# IBM Watson OpenScale Lab instructions

This notebook should be run in a Watson Studio project, using with **Python 3.6 with Spark** runtime environment. **If you are viewing this in Watson Studio and do not see Python 3.6 with Spark in the upper right corner of your screen, please update the runtime now.** It requires service credentials for the following Cloud services:
  * IBM Watson OpenScale
  * Watson Machine Learning
  
If you have a paid Cloud account, you may also provision a **Databases for PostgreSQL** or **Db2 Warehouse** service to take full advantage of integration with Watson Studio and continuous learning services. If you choose not to provision this paid service, you can use the free internal PostgreSQL storage with OpenScale, but will not be able to configure continuous learning for your model.

The notebook will train, create and deploy a Propensity to Buy Model, configure OpenScale to monitor that deployment, and inject seven days' worth of historical records and measurements for viewing in the OpenScale Insights dashboard.

# Business Scenario

National Chemical Company (NCC) operates two plants plants in South Lousiana.  Both plants produce the same chemical.  Both plants use the same suppliers.  Both plants are managed by the same people.  The only difference between the two plants is age.  Plant A is brand new.  It opened less than a year ago.  Plant B is more than 20 years old.

Bad Yields cost NCC millions of dollars each year.  A bad yield just means that the final product produced by the factory does not meet quality standards.  Ideally, they would like to predict a bad yield a few days in advance, drill down to see why there is a prediction of a bad yield and make corrections.  To accomplish this, NCC turned to IBM and Watson.  

In this notebook, you will construct a machine learning model and deploy that model to a Watson Machine Learning Service.  This model is based on historical data and predicts the probability of a bad yield.  

After the model is built and deployed, the notebook configures Watson Openscale.  Watson Openscale will allow NCC to monitor the accuracy of the model over-time and understand the key factors that cause  bad yields.  Knowing why a bad yield will occur allows the plant operators to make adjustments and prevent them from occuring.

Another key issue that NCC is worried about is the bad sensor readings in Plant B, the older plant.  They are concerned that the bad sensor readings will lead to invalid predictions.  Given that Plant A and B an should have a similar percentage of predicted Bad Yield yields, we will use the bias and fairness detection monitors inside Watson Openscale to correct bias that may occur because of the older sensors in Plant B.






# Package installation

In [58]:
!rm -rf $PIP_BUILD
!pip install psycopg2-binary | tail -n 1
!pip install --upgrade watson-machine-learning-client --no-cache | tail -n 1
!pip install --upgrade ibm-ai-openscale --no-cache | tail -n 1
!pip install --upgrade numpy --no-cache | tail -n 1
!pip install --upgrade lime --no-cache | tail -n 1
!pip install --upgrade SciPy --no-cache | tail -n 1

Target directory /home/spark/shared/user-libs/python3/psycopg2 already exists. Specify --upgrade to force replacement.
Target directory /home/spark/shared/user-libs/python3/psycopg2_binary-2.8.2.dist-info already exists. Specify --upgrade to force replacement.
tensorflow 1.13.1 requires tensorboard<1.14.0,>=1.13.0, which is not installed.
spyder 3.3.3 requires pyqt5<=5.12; python_version >= "3", which is not installed.
ibm-cos-sdk-core 2.4.4 has requirement urllib3<1.25,>=1.20, but you'll have urllib3 1.25.3 which is incompatible.
botocore 1.12.82 has requirement urllib3<1.25,>=1.20, but you'll have urllib3 1.25.3 which is incompatible.
Exception:
Traceback (most recent call last):
  File "/opt/ibm/conda/miniconda36/lib/python3.6/site-packages/pip/_internal/cli/base_command.py", line 176, in main
    status = self.run(options, args)
  File "/opt/ibm/conda/miniconda36/lib/python3.6/site-packages/pip/_internal/commands/install.py", line 441, in run
    options.target_dir, target_temp_dir

#### Provision services and configure credentials

If you have not already, provision an instance of IBM Watson OpenScale using the [OpenScale link in the Cloud catalog](https://cloud.ibm.com/catalog/services/ai-openscale).

Your Cloud API key can be generated by going to the [**Users** section of the Cloud console](https://cloud.ibm.com/iam#/users). From that page, click your name, scroll down to the **API Keys** section, and click **Create an IBM Cloud API key**. Give your key a name and click **Create**, then copy the created key and paste it below.

In [59]:
CLOUD_API_KEY = "Insert Cloud API Key"

Next you will need credentials for Watson Machine Learning. If you already have a WML instance, you may use credentials for it.  Find your existing instance of WML [here](https://cloud.ibm.com/resources).   To provision a new Lite instance of WML, use the [Cloud catalog](https://cloud.ibm.com/catalog/services/machine-learning), give your service a name, and click **Create**. Once your instance is created, click the **Service Credentials** link on the left side of the screen. Click the **New credential** button, give your credentials a name, and click **Add**. Your new credentials can be accessed by clicking the **View credentials** button. Copy and paste your WML credentials into the cell below.

In [60]:
WML_CREDENTIALS = {Insert WML Credentials}

This lab can use Databases for PostgreSQL, Db2 Warehouse, or a free internal verison of PostgreSQL to create a datamart for OpenScale.

If you have previously configured OpenScale, it will use your existing datamart, and not interfere with any models you are currently monitoring. Do not update the cell below.

If you do not have a paid Cloud account or would prefer not to provision this paid service, you may use the free internal PostgreSQL service with OpenScale. Do not update the cell below.

To provision a new instance of Db2 Warehouse, locate [Db2 Warehouse in the Cloud catalog](https://cloud.ibm.com/catalog/services/db2-warehouse), give your service a name, and click **Create**. Once your instance is created, click the **Service Credentials** link on the left side of the screen. Click the **New credential** button, give your credentials a name, and click **Add**. Your new credentials can be accessed by clicking the **View credentials** button. Copy and paste your Db2 Warehouse credentials into the cell below.

To provision a new instance of Databases for PostgreSQL, locate [Databases for PostgreSQL in the Cloud catalog](https://cloud.ibm.com/catalog/services/databases-for-postgresql), give your service a name, and click **Create**. Once your instance is created, click the **Service Credentials** link on the left side of the screen. Click the **New credential** button, give your credentials a name, and click **Add**. Your new credentials can be accessed by clicking the **View credentials** button. Copy and paste your Databases for PostgreSQL credentials into the cell below.

In [61]:
DB_CREDENTIALS = None

__If you previously configured OpenScale to use the free internal version of PostgreSQL, you can switch to a new datamart using a paid database service.__ If you would like to delete the internal PostgreSQL configuration and create a new one using service credentials supplied in the cell above, set the __KEEP_MY_INTERNAL_POSTGRES__ variable below to __False__ below. In this case, the notebook will remove your existing internal PostgreSQL datamart and create a new one with the supplied credentials. __*NO DATA MIGRATION WILL OCCUR.*__

In [62]:
KEEP_MY_INTERNAL_POSTGRES = True

# Run the notebook

At this point, the notebook is ready to run. You can either run the cells one at a time, or click the **Kernel** option above and select **Restart and Run All** to run all the cells.

# Load and explore data

## Load the training data from github

In [63]:
!rm df_training.csv
!wget https://raw.githubusercontent.com/shadgriffin/oglabworking/master/df_training.csv

--2019-06-06 14:17:12--  https://raw.githubusercontent.com/shadgriffin/oglabworking/master/df_training.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.48.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 340414 (332K) [text/plain]
Saving to: 'df_training.csv'

df_training.csv     100%[===================>] 332.44K  --.-KB/s    in 0.02s   

2019-06-06 14:17:13 (15.3 MB/s) - 'df_training.csv' saved [340414/340414]



In [64]:
from pyspark.sql import SparkSession
import pandas as pd
import json
import numpy as np

spark = SparkSession.builder.getOrCreate()
pd_data = pd.read_csv("df_training.csv", sep=",", header=0)
#df_data = spark.read.csv(path="df_training.csv", sep=",", header=True, inferSchema=True)
#df_data.head()

In [65]:
df_training=pd_data

In [66]:
pd_data.columns

Index(['WS_001_FLOW_MEAN', 'WS_001_FLOW_MIN', 'WS_001_FLOW_MAX',
       'WS_001_CONC_MEAN', 'WS_001_CONC_MIN', 'WS_001_CONC_MAX',
       'DMW_FLOW_MEAN', 'DMW_FLOW_MIN', 'DMW_FLOW_MAX', 'ALK_FLOW_MEAN',
       'ALK_FLOW_MIN', 'ALK_FLOW_MAX', 'RPM_MEAN', 'RPM_MIN', 'RPM_MAX',
       'PLANT_A', 'BAD_YIELD'],
      dtype='object')

In [67]:
pd_data['wookie']=(np.random.randint(1, 1000,pd_data.shape[0]))/1000


In [68]:
PLANT_B=pd_data[pd_data['PLANT_A'] ==0]
PLANT_A=pd_data[pd_data['PLANT_A']==1]
PLANT_B_0=PLANT_B[PLANT_B['BAD_YIELD']=='BAD']
PLANT_B_1=PLANT_B[PLANT_B['BAD_YIELD']=='GOOD']
#PLANT_B_1a=PLANT_B_1[PLANT_B_1['wookie']<=0.75]
#PLANT_B_1b=PLANT_B_1[PLANT_B_1['wookie']>0.75]



PLANT_B_1 = PLANT_B_1.copy()

PLANT_B_1['BAD_YIELD'] = np.where(PLANT_B_1['wookie']<=0.000001, 'BAD', 'GOOD')

In [69]:
pd_data = PLANT_B_0.append([PLANT_B_1, PLANT_A])

pd_data=pd_data.drop(columns=['wookie'])

In [70]:
pd.crosstab(pd_data.PLANT_A, pd_data.BAD_YIELD).apply(lambda r: r/r.sum(), axis = 1)

BAD_YIELD,BAD,GOOD
PLANT_A,,
0,0.472175,0.527825
1,0.200426,0.799574


In [71]:
pd_data.shape
pd_data.head()

,WS_001_FLOW_MEAN,WS_001_FLOW_MIN,WS_001_FLOW_MAX,WS_001_CONC_MEAN,WS_001_CONC_MIN,WS_001_CONC_MAX,DMW_FLOW_MEAN,DMW_FLOW_MIN,DMW_FLOW_MAX,ALK_FLOW_MEAN,ALK_FLOW_MIN,ALK_FLOW_MAX,RPM_MEAN,RPM_MIN,RPM_MAX,PLANT_A,BAD_YIELD
0,87.226264,87.462261,87.192360,62.998835,62.8255,63.1551,92.682929,92.687150,92.67134,87.957784,88.105276,87.929657,5010.885417,5010.67,5011.11,0,BAD
1,87.061707,86.429871,87.104682,64.899764,60.0440,65.0691,92.930550,92.934958,92.91834,87.847303,87.127372,87.929657,4997.844915,4319.54,5009.69,0,BAD
13,87.230540,87.462261,87.192360,64.999974,64.8091,65.1523,92.930550,92.934958,92.91834,87.957784,88.105276,87.929657,5009.466250,5008.93,5009.94,0,BAD
16,87.085769,87.204164,87.017004,65.001700,64.7714,65.2815,92.930550,92.934958,92.91834,86.453545,86.638420,86.425383,5008.098264,5002.25,5010.52,0,BAD
30,87.213436,87.376229,87.104682,64.998859,64.8578,65.1560,92.930550,92.934958,92.91834,86.453545,86.638420,86.425383,5010.489097,5010.30,5010.75,0,BAD


In [72]:
df_data = spark.createDataFrame(pd_data)
df_data.head()

Row(WS_001_FLOW_MEAN=87.226264, WS_001_FLOW_MIN=87.462261, WS_001_FLOW_MAX=87.19236, WS_001_CONC_MEAN=62.998835, WS_001_CONC_MIN=62.8255, WS_001_CONC_MAX=63.1551, DMW_FLOW_MEAN=92.682929, DMW_FLOW_MIN=92.68715, DMW_FLOW_MAX=92.67134, ALK_FLOW_MEAN=87.957784, ALK_FLOW_MIN=88.105276, ALK_FLOW_MAX=87.92965699999998, RPM_MEAN=5010.885417, RPM_MIN=5010.67, RPM_MAX=5011.11, PLANT_A=0, BAD_YIELD='BAD')

## Explore data

In [73]:
df_data.printSchema()

root
 |-- WS_001_FLOW_MEAN: double (nullable = true)
 |-- WS_001_FLOW_MIN: double (nullable = true)
 |-- WS_001_FLOW_MAX: double (nullable = true)
 |-- WS_001_CONC_MEAN: double (nullable = true)
 |-- WS_001_CONC_MIN: double (nullable = true)
 |-- WS_001_CONC_MAX: double (nullable = true)
 |-- DMW_FLOW_MEAN: double (nullable = true)
 |-- DMW_FLOW_MIN: double (nullable = true)
 |-- DMW_FLOW_MAX: double (nullable = true)
 |-- ALK_FLOW_MEAN: double (nullable = true)
 |-- ALK_FLOW_MIN: double (nullable = true)
 |-- ALK_FLOW_MAX: double (nullable = true)
 |-- RPM_MEAN: double (nullable = true)
 |-- RPM_MIN: double (nullable = true)
 |-- RPM_MAX: double (nullable = true)
 |-- PLANT_A: long (nullable = true)
 |-- BAD_YIELD: string (nullable = true)



In [74]:
print("Number of records: " + str(df_data.count()))

Number of records: 2000


# Create a model

In [75]:
spark_df = df_data
(train_data, test_data) = spark_df.randomSplit([0.8, 0.2], 24)

MODEL_NAME = "Yield Model"
DEPLOYMENT_NAME = "Yield Model"

print("Number of records for training: " + str(train_data.count()))
print("Number of records for evaluation: " + str(test_data.count()))

spark_df.printSchema()

Number of records for training: 1612
Number of records for evaluation: 388
root
 |-- WS_001_FLOW_MEAN: double (nullable = true)
 |-- WS_001_FLOW_MIN: double (nullable = true)
 |-- WS_001_FLOW_MAX: double (nullable = true)
 |-- WS_001_CONC_MEAN: double (nullable = true)
 |-- WS_001_CONC_MIN: double (nullable = true)
 |-- WS_001_CONC_MAX: double (nullable = true)
 |-- DMW_FLOW_MEAN: double (nullable = true)
 |-- DMW_FLOW_MIN: double (nullable = true)
 |-- DMW_FLOW_MAX: double (nullable = true)
 |-- ALK_FLOW_MEAN: double (nullable = true)
 |-- ALK_FLOW_MIN: double (nullable = true)
 |-- ALK_FLOW_MAX: double (nullable = true)
 |-- RPM_MEAN: double (nullable = true)
 |-- RPM_MIN: double (nullable = true)
 |-- RPM_MAX: double (nullable = true)
 |-- PLANT_A: long (nullable = true)
 |-- BAD_YIELD: string (nullable = true)



In [76]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, IndexToString, VectorAssembler
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml import Pipeline, Model
from pyspark.ml import linalg



In [77]:
si_Label = StringIndexer(inputCol="BAD_YIELD", outputCol="label").fit(spark_df)
label_converter = IndexToString(inputCol="prediction", outputCol="predictedLabel", labels=si_Label.labels)

In [78]:
pd_data.columns

Index(['WS_001_FLOW_MEAN', 'WS_001_FLOW_MIN', 'WS_001_FLOW_MAX',
       'WS_001_CONC_MEAN', 'WS_001_CONC_MIN', 'WS_001_CONC_MAX',
       'DMW_FLOW_MEAN', 'DMW_FLOW_MIN', 'DMW_FLOW_MAX', 'ALK_FLOW_MEAN',
       'ALK_FLOW_MIN', 'ALK_FLOW_MAX', 'RPM_MEAN', 'RPM_MIN', 'RPM_MAX',
       'PLANT_A', 'BAD_YIELD'],
      dtype='object')

In [79]:
va_features = VectorAssembler(inputCols=['WS_001_FLOW_MEAN', 'WS_001_FLOW_MIN', 'WS_001_FLOW_MAX',
       'WS_001_CONC_MEAN', 'WS_001_CONC_MIN', 'WS_001_CONC_MAX',
       'DMW_FLOW_MEAN', 'DMW_FLOW_MIN', 'DMW_FLOW_MAX', 'ALK_FLOW_MEAN',
       'ALK_FLOW_MIN', 'ALK_FLOW_MAX', 'RPM_MEAN', 'RPM_MIN', 'RPM_MAX', 'PLANT_A'], outputCol="features")

In [80]:
from pyspark.ml.classification import RandomForestClassifier
classifier = RandomForestClassifier(featuresCol="features")

pipeline = Pipeline(stages=[ si_Label, va_features, classifier, label_converter])
model = pipeline.fit(train_data)

In [81]:
predictions = model.transform(test_data)
evaluatorDT = BinaryClassificationEvaluator(rawPredictionCol="prediction")
area_under_curve = evaluatorDT.evaluate(predictions)

#default evaluation is areaUnderROC
print("areaUnderROC = %g" % area_under_curve)

areaUnderROC = 0.944563


# Save and deploy the model

from scipy import sparse
from scipy import linalg

In [82]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient 
import json 



wml_client = WatsonMachineLearningAPIClient(WML_CREDENTIALS)

### Remove existing model and deployment

In [83]:
model_deployment_ids = wml_client.deployments.get_uids()
for deployment_id in model_deployment_ids:
    deployment = wml_client.deployments.get_details(deployment_id)
    model_id = deployment['entity']['deployable_asset']['guid']
    if deployment['entity']['name'] == DEPLOYMENT_NAME:
        print('Deleting deployment id', deployment_id)
        wml_client.deployments.delete(deployment_id)
        print('Deleting model id', model_id)
        wml_client.repository.delete(model_id)
wml_client.repository.list_models()

Deleting deployment id ee1dbb19-702a-4b82-941e-d1ddfce34509
Deleting model id 6a2667eb-f748-4924-80eb-c9b761549905
----  ----  -------  ---------
GUID  NAME  CREATED  FRAMEWORK
----  ----  -------  ---------


In [84]:
model_props = {
    wml_client.repository.ModelMetaNames.NAME: "{}".format(MODEL_NAME),
    wml_client.repository.ModelMetaNames.EVALUATION_METHOD: "binary",
    wml_client.repository.ModelMetaNames.EVALUATION_METRICS: [
        {
           "name": "areaUnderROC",
           "value": area_under_curve,
           "threshold": 0.85
        }
    ]
}

In [85]:
wml_models = wml_client.repository.get_details()
model_uid = None
for model_in in wml_models['models']['resources']:
    if MODEL_NAME == model_in['entity']['name']:
        model_uid = model_in['metadata']['guid']
        break

if model_uid is None:
    print("Storing model ...")

    published_model_details = wml_client.repository.store_model(model=model, meta_props=model_props, training_data=train_data, pipeline=pipeline)
    model_uid = wml_client.repository.get_model_uid(published_model_details)
    print("Done")

Storing model ...


/home/spark/shared/user-libs/python3/urllib3/connectionpool.py:851: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/home/spark/shared/user-libs/python3/urllib3/connectionpool.py:851: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/home/spark/shared/user-libs/python3/urllib3/connectionpool.py:851: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/home/spark/shared/user-libs/python3/urllib3/connectionpool.py:851: InsecureRequestWarning: Unverified HTTPS request is being made

Done


In [86]:
model_uid

'3437f090-aff0-49d9-b062-08b0550882fa'

In [87]:
wml_deployments = wml_client.deployments.get_details()
deployment_uid = None
for deployment in wml_deployments['resources']:
    if DEPLOYMENT_NAME == deployment['entity']['name']:
        deployment_uid = deployment['metadata']['guid']
        break

if deployment_uid is None:
    print("Deploying model...")

    deployment = wml_client.deployments.create(artifact_uid=model_uid, name=DEPLOYMENT_NAME, asynchronous=False)
    deployment_uid = wml_client.deployments.get_uid(deployment)
    
print("Model id: {}".format(model_uid))
print("Deployment id: {}".format(deployment_uid))

Deploying model...


#######################################################################################

Synchronous deployment creation for uid: '3437f090-aff0-49d9-b062-08b0550882fa' started

#######################################################################################


INITIALIZING
DEPLOY_SUCCESS


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='2b37af4c-7b67-41ff-9403-1b6d33a5c755'
------------------------------------------------------------------------------------------------


Model id: 3437f090-aff0-49d9-b062-08b0550882fa
Deployment id: 2b37af4c-7b67-41ff-9403-1b6d33a5c755


# Configure OpenScale

In [88]:
from ibm_ai_openscale import APIClient
from ibm_ai_openscale.engines import *
from ibm_ai_openscale.utils import *
from ibm_ai_openscale.supporting_classes import PayloadRecord, Feature
from ibm_ai_openscale.supporting_classes.enums import *

### Get AI OpenScale GUID

In [89]:
import requests

AIOS_GUID = None
token_data = {
    'grant_type': 'urn:ibm:params:oauth:grant-type:apikey',
    'response_type': 'cloud_iam',
    'apikey': CLOUD_API_KEY
}

response = requests.post('https://iam.bluemix.net/identity/token', data=token_data)
iam_token = response.json()['access_token']
iam_headers = {
    'Content-Type': 'application/json',
    'Authorization': 'Bearer %s' % iam_token
}

resources = json.loads(requests.get('https://resource-controller.cloud.ibm.com/v2/resource_instances', headers=iam_headers).text)['resources']
for resource in resources:
    if "aiopenscale" in resource['id'].lower():
        AIOS_GUID = resource['guid']
        
AIOS_CREDENTIALS = {
    "instance_guid": AIOS_GUID,
    "apikey": CLOUD_API_KEY,
    "url": "https://api.aiopenscale.cloud.ibm.com"
}

if AIOS_GUID is None:
    print('AI OpenScale GUID NOT FOUND')
else:
    print(AIOS_GUID)

8cb0facb-0ece-42f4-a78f-1ff08406b717


## Create schema and datamart

In [90]:
ai_client = APIClient(aios_credentials=AIOS_CREDENTIALS)
ai_client.version
time.sleep(20)

### Set up datamart

In [91]:
try:
    data_mart_details = ai_client.data_mart.get_details()
    if 'internal_database' in data_mart_details and data_mart_details['internal_database']:
        if KEEP_MY_INTERNAL_POSTGRES:
            print('Using existing internal datamart.')
        else:
            if DB_CREDENTIALS is None:
                print('No postgres credentials supplied. Using existing internal datamart')
            else:
                print('Switching to external datamart')
                ai_client.data_mart.delete(force=True)
                ai_client.data_mart.setup(db_credentials=DB_CREDENTIALS)
    else:
        print('Using existing external datamart')
except:
    if DB_CREDENTIALS is None:
        print('Setting up internal datamart')
        ai_client.data_mart.setup(internal_db=True)
    else:
        print('Setting up external datamart')
        ai_client.data_mart.setup(db_credentials=DB_CREDENTIALS)
    

Setting up internal datamart


In [92]:
data_mart_details = ai_client.data_mart.get_details()
data_mart_details

{'internal_database': True,
 'service_instance_crn': 'crn:v1:bluemix:public:aiopenscale:us-south:a/97deeb0b7e78431438a00a04f20580b7:8cb0facb-0ece-42f4-a78f-1ff08406b717::',
 'status': {'state': 'active'},
 'database_configuration': {},
 'internal_database_pool': 'compose-psql'}

## Bind machine learning engines

In [93]:
binding_uid = ai_client.data_mart.bindings.add('WML instance', WatsonMachineLearningInstance(WML_CREDENTIALS))
if binding_uid is None:
    binding_uid = ai_client.data_mart.bindings.get_details()['service_bindings'][0]['metadata']['guid']
bindings_details = ai_client.data_mart.bindings.get_details()
ai_client.data_mart.bindings.list()

5a8c208d-b89f-4fcd-af26-c3cd4a48ac01,WML instance,watson_machine_learning,2019-06-06T14:18:02.309Z


In [94]:
print(binding_uid)

5a8c208d-b89f-4fcd-af26-c3cd4a48ac01


In [95]:
ai_client.data_mart.bindings.list_assets()

3437f090-aff0-49d9-b062-08b0550882fa,Yield Model,2019-06-06T14:17:30.406Z,model,mllib-2.3,5a8c208d-b89f-4fcd-af26-c3cd4a48ac01,False


## Subscriptions

### Remove existing  subscriptions

In [96]:
subscriptions_uids = ai_client.data_mart.subscriptions.get_uids()
for subscription in subscriptions_uids:
    sub_name = ai_client.data_mart.subscriptions.get_details(subscription)['entity']['asset']['name']
    if sub_name == MODEL_NAME:
        ai_client.data_mart.subscriptions.delete(subscription)
        print('Deleted existing subscription for', MODEL_NAME)

In [97]:
subscription = ai_client.data_mart.subscriptions.add(WatsonMachineLearningAsset(
    model_uid,
    problem_type=ProblemType.BINARY_CLASSIFICATION,
    input_data_type=InputDataType.STRUCTURED,
    label_column='BAD_YIELD',
    prediction_column='predictedLabel',
    probability_column='probability',
    feature_columns = ['WS_001_FLOW_MEAN', 'WS_001_FLOW_MIN', 'WS_001_FLOW_MAX',
       'WS_001_CONC_MEAN', 'WS_001_CONC_MIN', 'WS_001_CONC_MAX',
       'DMW_FLOW_MEAN', 'DMW_FLOW_MIN', 'DMW_FLOW_MAX', 'ALK_FLOW_MEAN',
       'ALK_FLOW_MIN', 'ALK_FLOW_MAX', 'RPM_MEAN', 'RPM_MIN', 'RPM_MAX','PLANT_A'],
    categorical_columns = ['PLANT_A']
))

if subscription is None:
    print('Subscription already exists; get the existing one')
    subscriptions_uids = ai_client.data_mart.subscriptions.get_uids()
    for sub in subscriptions_uids:
        if ai_client.data_mart.subscriptions.get_details(sub)['entity']['asset']['name'] == MODEL_NAME:
            subscription = ai_client.data_mart.subscriptions.get(sub)

Get subscription list

In [98]:
subscriptions_uids = ai_client.data_mart.subscriptions.get_uids()
ai_client.data_mart.subscriptions.list()

3437f090-aff0-49d9-b062-08b0550882fa,Yield Model,model,5a8c208d-b89f-4fcd-af26-c3cd4a48ac01,2019-06-06T14:18:04.150Z


In [99]:

subscription.get_details()

{'metadata': {'guid': '3437f090-aff0-49d9-b062-08b0550882fa',
  'url': '/v1/data_marts/8cb0facb-0ece-42f4-a78f-1ff08406b717/service_bindings/5a8c208d-b89f-4fcd-af26-c3cd4a48ac01/subscriptions/3437f090-aff0-49d9-b062-08b0550882fa',
  'created_at': '2019-06-06T14:18:04.150Z',
  'modified_at': '2019-06-06T14:18:04.797Z'},
 'entity': {'service_binding_id': '5a8c208d-b89f-4fcd-af26-c3cd4a48ac01',
  'asset_properties': {'runtime_environment': 'spark-2.3',
   'predicted_target_field': 'predictedLabel',
   'training_data_schema': {'type': 'struct',
    'fields': [{'name': 'WS_001_FLOW_MEAN',
      'type': 'double',
      'nullable': True,
      'metadata': {'modeling_role': 'feature'}},
     {'name': 'WS_001_FLOW_MIN',
      'type': 'double',
      'nullable': True,
      'metadata': {'modeling_role': 'feature'}},
     {'name': 'WS_001_FLOW_MAX',
      'type': 'double',
      'nullable': True,
      'metadata': {'modeling_role': 'feature'}},
     {'name': 'WS_001_CONC_MEAN',
      'type': 'dou

### Score the model so we can configure monitors

In [100]:
propensity_to_buy_scoring_endpoint = None
print(deployment_uid)

for deployment in wml_client.deployments.get_details()['resources']:
    if deployment_uid in deployment['metadata']['guid']:
        propensity_to_buy_scoring_endpoint = deployment['entity']['scoring_url']
        
print(propensity_to_buy_scoring_endpoint)

2b37af4c-7b67-41ff-9403-1b6d33a5c755
https://us-south.ml.cloud.ibm.com/v3/wml_instances/5a8c208d-b89f-4fcd-af26-c3cd4a48ac01/deployments/2b37af4c-7b67-41ff-9403-1b6d33a5c755/online


In [101]:
fields = ['WS_001_FLOW_MEAN', 'WS_001_FLOW_MIN', 'WS_001_FLOW_MAX',
       'WS_001_CONC_MEAN', 'WS_001_CONC_MIN', 'WS_001_CONC_MAX',
       'DMW_FLOW_MEAN', 'DMW_FLOW_MIN', 'DMW_FLOW_MAX', 'ALK_FLOW_MEAN',
       'ALK_FLOW_MIN', 'ALK_FLOW_MAX', 'RPM_MEAN', 'RPM_MIN', 'RPM_MAX', 'PLANT_A']
values = [[91.472265,91.591823,91.400894,29.999306,29.9453,30.0347,88.597187,88.598325,88.595852,91.467674,91.52794,91.439629,5010.304375,5010.05,5010.61,1],
          [91.286568,91.333725,91.225539,29.999756,29.8993,30.1047,88.597187,88.598325,88.595852,91.467674,91.52794,91.439629,5011.043681,5010.74,5011.32,1],
          [91.581607,91.677855,91.57625,30.000624,29.9481,30.0648,88.597187,88.598325,88.595852,91.467674,91.52794,91.439629,5008.680625,5007.45,5009.42,0],
          [91.332992,91.419757,91.313216,30.000023,29.9221,30.1148,88.597187,88.598325,88.595852,91.467674,91.52794,91.439629,5010.7575,5010.47,5011.09,1],
          [87.178007,87.376229,87.19236,65.001133,64.8447,65.1378,92.93055,92.934958,92.91834,87.957784,88.105276,87.929657,5009.978056,5009.61,5010.3,1],
          [91.463103,91.591823,91.400894,29.999842,29.9367,30.0449,88.597187,88.598325,88.595852,91.467674,91.52794,91.439629,5010.263889,5009.75,5010.5,0],
          [87.284294,87.462261,87.280037,65.000094,64.9171,65.0708,92.93055,92.934958,92.91834,87.957784,88.105276,87.929657,5008.015347,5006.6,5009.73,1],
          [87.121198,87.290196,87.104682,65.000774,64.8889,65.1762,92.93055,92.934958,92.91834,86.453545,86.63842,86.425383,5009.210625,5008.67,5009.67,1],
          [87.2452,87.462261,87.19236,62.999506,62.8072,63.167,92.682929,92.68715,92.67134,87.957784,88.105276,87.929657,5011.408889,5011.12,5011.59,0],
          [87.099207,87.032099,87.280037,64.983894,62.8779,65.1535,92.93055,92.934958,92.91834,86.453545,86.63842,86.425383,5010.101181,5009.21,5010.83,0],
          [91.243197,91.333725,91.225539,29.998703,29.9305,30.1019,88.597187,88.598325,88.595852,91.467674,91.52794,91.439629,5010.714444,5010.44,5011.11,1],
          [91.612149,91.677855,91.57625,29.998469,29.9674,30.0269,88.597187,88.598325,88.595852,91.467674,91.52794,91.439629,5009.607431,5008.38,5010.55,1],
          [87.005748,87.204164,86.929326,65.001815,64.1377,65.9022,92.93055,92.934958,92.91834,86.453545,86.63842,86.425383,5010.824861,5010.53,5011.05,1],
          [91.260301,91.333725,91.225539,30.000992,29.9268,30.0831,88.597187,88.598325,88.595852,91.467674,91.52794,91.439629,5010.054444,5009.72,5010.31,1],
          [90.70443,90.731497,90.699472,29.513889,29.0,30.0,88.597187,88.598325,88.595852,90.464848,90.550036,90.43678,5007.784861,5006.61,5008.85,1],
          [91.538237,91.677855,91.488572,30.000409,29.9454,30.0518,88.597187,88.598325,88.595852,91.467674,91.52794,91.439629,5009.791319,5008.99,5010.11,1],
          [87.051561,87.204164,87.017004,64.999602,64.8108,65.146,92.93055,92.934958,92.91834,86.453545,86.63842,86.425383,5009.726389,5005.68,5010.44,0],
          [91.614593,91.763888,91.57625,29.999788,29.9509,30.0482,88.597187,88.598325,88.595852,91.467674,91.52794,91.439629,5008.214444,5006.21,5009.69,0],
          [91.365367,91.419757,91.313216,30.00108,29.8762,30.1167,88.597187,88.598325,88.595852,91.467674,91.52794,91.439629,4998.986875,4996.14,5003.11,0]
         ]
payload_scoring = {"fields": fields,"values": values}
scoring_response = wml_client.deployments.score(propensity_to_buy_scoring_endpoint, payload_scoring)

print(scoring_response)

{'fields': ['WS_001_FLOW_MEAN', 'WS_001_FLOW_MIN', 'WS_001_FLOW_MAX', 'WS_001_CONC_MEAN', 'WS_001_CONC_MIN', 'WS_001_CONC_MAX', 'DMW_FLOW_MEAN', 'DMW_FLOW_MIN', 'DMW_FLOW_MAX', 'ALK_FLOW_MEAN', 'ALK_FLOW_MIN', 'ALK_FLOW_MAX', 'RPM_MEAN', 'RPM_MIN', 'RPM_MAX', 'PLANT_A', 'BAD_YIELD', 'label', 'features', 'rawPrediction', 'probability', 'prediction', 'predictedLabel'], 'values': [[91.472265, 91.591823, 91.400894, 29.999306, 29.9453, 30.0347, 88.597187, 88.598325, 88.595852, 91.467674, 91.52794, 91.439629, 5010.304375, 5010.05, 5010.61, 1, 'GOOD', 0.0, [91.472265, 91.591823, 91.400894, 29.999306, 29.9453, 30.0347, 88.597187, 88.598325, 88.595852, 91.467674, 91.52794, 91.439629, 5010.304375, 5010.05, 5010.61, 1.0], [19.82363347572176, 0.17636652427824395], [0.9911816737860878, 0.008818326213912197], 0.0, 'GOOD'], [91.286568, 91.333725, 91.225539, 29.999756, 29.8993, 30.1047, 88.597187, 88.598325, 88.595852, 91.467674, 91.52794, 91.439629, 5011.043681, 5010.74, 5011.32, 1, 'GOOD', 0.0, [91.

## Quality and feedback monitoring

### Enable quality monitoring

Wait ten seconds to allow the payload logging table to be set up before we begin enabling monitors.

In [102]:
time.sleep(20)
subscription.quality_monitoring.enable(threshold=0.7, min_records=100)

### Feedback logging

In [103]:
!rm df_feedback.json
!wget https://raw.githubusercontent.com/shadgriffin/oglabworking/master/df_feedback.json

--2019-06-06 14:18:27--  https://raw.githubusercontent.com/shadgriffin/oglabworking/master/df_feedback.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.48.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 151790 (148K) [text/plain]
Saving to: 'df_feedback.json'

df_feedback.json    100%[===================>] 148.23K  --.-KB/s    in 0.03s   

2019-06-06 14:18:27 (4.90 MB/s) - 'df_feedback.json' saved [151790/151790]



In [104]:
with open('df_feedback.json') as feedback_file:
    df_feedback = json.load(feedback_file)
subscription.feedback_logging.store(df_feedback)

In [105]:
subscription.feedback_logging.show_table()

87.203662,87.376229,87.19236,63.000028,62.7366,63.2387,92.682929,92.68715,92.67134,87.957784,88.105276,87.929657,5011.305417,5011.04,5011.62,1,GOOD,2019-06-06 14:18:28.508407+00:00
91.463103,91.591823,91.400894,30.000738,29.9452,30.0657,88.597187,88.598325,88.595852,91.467674,91.52794,91.439629,5009.280486,5009.02,5009.51,1,GOOD,2019-06-06 14:18:28.508407+00:00
87.079049,87.290196,87.017004,65.000127,64.8321,65.1673,92.93055,92.934958,92.91834,86.453545,86.63842,86.425383,5010.689444,5010.33,5010.87,0,BAD,2019-06-06 14:18:28.508407+00:00
91.621312,91.677855,91.57625,29.998649,29.9375,30.0415,88.597187,88.598325,88.595852,91.467674,91.52794,91.439629,5006.168611,5001.83,5008.77,0,GOOD,2019-06-06 14:18:28.508407+00:00
91.612149,91.677855,91.57625,29.998469,29.9674,30.0269,88.597187,88.598325,88.595852,91.467674,91.52794,91.439629,5009.607431,5008.38,5010.55,1,GOOD,2019-06-06 14:18:28.508407+00:00
91.442334,91.50579,91.488572,29.999469,29.9075,30.1057,88.597187,88.598325,88.595852,91.467674,91.52794,91.439629,5008.133472,5006.6,5008.89,1,GOOD,2019-06-06 14:18:28.508407+00:00
91.28107,91.419757,91.225539,30.000265,29.9206,30.0748,88.597187,88.598325,88.595852,91.467674,91.52794,91.439629,5009.936389,5009.5,5010.51,1,GOOD,2019-06-06 14:18:28.508407+00:00
91.37453,91.419757,91.400894,30.000354,29.955,30.0593,88.597187,88.598325,88.595852,91.467674,91.52794,91.439629,4999.280278,4994.91,5003.84,1,GOOD,2019-06-06 14:18:28.508407+00:00
87.297122,87.462261,87.280037,64.997488,64.9247,65.0731,92.93055,92.934958,92.91834,87.957784,88.105276,87.929657,5006.536875,5004.3,5008.65,1,GOOD,2019-06-06 14:18:28.508407+00:00
91.570612,91.677855,91.57625,30.001588,29.9586,30.0482,88.597187,88.598325,88.595852,91.467674,91.52794,91.439629,5007.931667,5006.93,5008.58,1,GOOD,2019-06-06 14:18:28.508407+00:00


In [106]:
run_details = subscription.quality_monitoring.run()
status = run_details['status']
id = run_details['id']
print(id)

print("Run status: {}".format(status))

start_time = time.time()
elapsed_time = 0

while status != 'completed' and elapsed_time < 60:
    time.sleep(10)
    run_details = subscription.quality_monitoring.get_run_details(run_uid=id)
    status = run_details['status']
    elapsed_time = time.time() - start_time
    print("Run status: {}".format(status))

58ecfbf7-6a15-4463-bdb2-2e6278df646a
Run status: initializing
Run status: running
Run status: completed


In [107]:
subscription.quality_monitoring.get_run_details()

{'evaluations': [{'stages': [{'name': 'Prerequisite Check',
     'completed_at': '2019-06-06T14:18:32.973Z',
     'started_at': '2019-06-06T14:18:32.682Z',
     'id': 1,
     'properties': {'training_columns': ['WS_001_FLOW_MEAN',
       'WS_001_FLOW_MIN',
       'WS_001_FLOW_MAX',
       'WS_001_CONC_MEAN',
       'WS_001_CONC_MIN',
       'WS_001_CONC_MAX',
       'DMW_FLOW_MEAN',
       'DMW_FLOW_MIN',
       'DMW_FLOW_MAX',
       'ALK_FLOW_MEAN',
       'ALK_FLOW_MIN',
       'ALK_FLOW_MAX',
       'RPM_MEAN',
       'RPM_MIN',
       'RPM_MAX',
       'PLANT_A',
       'BAD_YIELD'],
      'input_columns': ['WS_001_FLOW_MEAN',
       'WS_001_FLOW_MIN',
       'WS_001_FLOW_MAX',
       'WS_001_CONC_MEAN',
       'WS_001_CONC_MIN',
       'WS_001_CONC_MAX',
       'DMW_FLOW_MEAN',
       'DMW_FLOW_MIN',
       'DMW_FLOW_MAX',
       'ALK_FLOW_MEAN',
       'ALK_FLOW_MIN',
       'ALK_FLOW_MAX',
       'RPM_MEAN',
       'RPM_MIN',
       'RPM_MAX',
       'PLANT_A']}},
    {'name': 

In [108]:
subscription.quality_monitoring.show_table()

2019-06-06 14:18:32.682000+00:00,true_positive_rate,89802a54-0f9d-4482-a984-a4419f3397c0,0.933579335793358,,,model_type: original,5a8c208d-b89f-4fcd-af26-c3cd4a48ac01,3437f090-aff0-49d9-b062-08b0550882fa,2b37af4c-7b67-41ff-9403-1b6d33a5c755
2019-06-06 14:18:32.682000+00:00,area_under_roc,89802a54-0f9d-4482-a984-a4419f3397c0,0.9578733441655404,0.7,,model_type: original,5a8c208d-b89f-4fcd-af26-c3cd4a48ac01,3437f090-aff0-49d9-b062-08b0550882fa,2b37af4c-7b67-41ff-9403-1b6d33a5c755
2019-06-06 14:18:32.682000+00:00,precision,89802a54-0f9d-4482-a984-a4419f3397c0,0.9511278195488722,,,model_type: original,5a8c208d-b89f-4fcd-af26-c3cd4a48ac01,3437f090-aff0-49d9-b062-08b0550882fa,2b37af4c-7b67-41ff-9403-1b6d33a5c755
2019-06-06 14:18:32.682000+00:00,f1_measure,89802a54-0f9d-4482-a984-a4419f3397c0,0.9422718808193669,,,model_type: original,5a8c208d-b89f-4fcd-af26-c3cd4a48ac01,3437f090-aff0-49d9-b062-08b0550882fa,2b37af4c-7b67-41ff-9403-1b6d33a5c755
2019-06-06 14:18:32.682000+00:00,accuracy,89802a54-0f9d-4482-a984-a4419f3397c0,0.969,,,model_type: original,5a8c208d-b89f-4fcd-af26-c3cd4a48ac01,3437f090-aff0-49d9-b062-08b0550882fa,2b37af4c-7b67-41ff-9403-1b6d33a5c755
2019-06-06 14:18:32.682000+00:00,log_loss,89802a54-0f9d-4482-a984-a4419f3397c0,0.07637163959866256,,,model_type: original,5a8c208d-b89f-4fcd-af26-c3cd4a48ac01,3437f090-aff0-49d9-b062-08b0550882fa,2b37af4c-7b67-41ff-9403-1b6d33a5c755
2019-06-06 14:18:32.682000+00:00,false_positive_rate,89802a54-0f9d-4482-a984-a4419f3397c0,0.01783264746227709,,,model_type: original,5a8c208d-b89f-4fcd-af26-c3cd4a48ac01,3437f090-aff0-49d9-b062-08b0550882fa,2b37af4c-7b67-41ff-9403-1b6d33a5c755
2019-06-06 14:18:32.682000+00:00,area_under_pr,89802a54-0f9d-4482-a984-a4419f3397c0,0.9285405487889465,,,model_type: original,5a8c208d-b89f-4fcd-af26-c3cd4a48ac01,3437f090-aff0-49d9-b062-08b0550882fa,2b37af4c-7b67-41ff-9403-1b6d33a5c755
2019-06-06 14:18:32.682000+00:00,recall,89802a54-0f9d-4482-a984-a4419f3397c0,0.933579335793358,,,model_type: original,5a8c208d-b89f-4fcd-af26-c3cd4a48ac01,3437f090-aff0-49d9-b062-08b0550882fa,2b37af4c-7b67-41ff-9403-1b6d33a5c755


subscription.quality_monitoring._get_data_from_rest_api()

In [109]:
ai_client.data_mart.get_deployment_metrics()

{'deployment_metrics': [{'subscription': {'subscription_id': '3437f090-aff0-49d9-b062-08b0550882fa',
    'url': '/v1/data_marts/8cb0facb-0ece-42f4-a78f-1ff08406b717/service_bindings/5a8c208d-b89f-4fcd-af26-c3cd4a48ac01/subscriptions/3437f090-aff0-49d9-b062-08b0550882fa'},
   'asset': {'name': 'Yield Model',
    'asset_id': '3437f090-aff0-49d9-b062-08b0550882fa',
    'url': 'https://us-south.ml.cloud.ibm.com/v3/wml_instances/5a8c208d-b89f-4fcd-af26-c3cd4a48ac01/published_models/3437f090-aff0-49d9-b062-08b0550882fa',
    'asset_type': 'model',
    'created_at': '2019-06-06T14:17:30.406Z'},
   'deployment': {'name': 'Yield Model',
    'url': 'https://us-south.ml.cloud.ibm.com/v3/wml_instances/5a8c208d-b89f-4fcd-af26-c3cd4a48ac01/deployments/2b37af4c-7b67-41ff-9403-1b6d33a5c755',
    'deployment_type': 'online',
    'scoring_endpoint': {'url': 'https://us-south.ml.cloud.ibm.com/v3/wml_instances/5a8c208d-b89f-4fcd-af26-c3cd4a48ac01/deployments/2b37af4c-7b67-41ff-9403-1b6d33a5c755/online',
 

## Fairness monitoring

In [110]:
subscription.fairness_monitoring.enable(
            features=[
                Feature("PLANT_A", majority=[[1,1]], minority=[[0,0]], threshold=0.95)
            ],
            favourable_classes=['GOOD'],
            unfavourable_classes=['BAD'],
            min_records=1000,
            training_data=df_training
        )

## Score the model again now that monitoring is configured

In [111]:
!rm df_payload_biased.json
!wget https://raw.githubusercontent.com/shadgriffin/oglabworking/master/df_payload_biased.json

--2019-06-06 14:18:54--  https://raw.githubusercontent.com/shadgriffin/oglabworking/master/df_payload_biased.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.48.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 290338 (284K) [text/plain]
Saving to: 'df_payload_biased.json'

df_payload_biased.j 100%[===================>] 283.53K  --.-KB/s    in 0.02s   

2019-06-06 14:18:55 (17.6 MB/s) - 'df_payload_biased.json' saved [290338/290338]



Score 1000 randomly chosen records

In [112]:
import random

with open('df_payload_biased.json', 'r') as scoring_file:
    scoring_data = json.load(scoring_file)

fields = scoring_data['fields']
values = []
for _ in range(1000):
    values.append(random.choice(scoring_data['values']))
payload_scoring = {"fields": fields, "values": values}

scoring_response = wml_client.deployments.score(propensity_to_buy_scoring_endpoint, payload_scoring)
print(scoring_response)

{'fields': ['WS_001_FLOW_MEAN', 'WS_001_FLOW_MIN', 'WS_001_FLOW_MAX', 'WS_001_CONC_MEAN', 'WS_001_CONC_MIN', 'WS_001_CONC_MAX', 'DMW_FLOW_MEAN', 'DMW_FLOW_MIN', 'DMW_FLOW_MAX', 'ALK_FLOW_MEAN', 'ALK_FLOW_MIN', 'ALK_FLOW_MAX', 'RPM_MEAN', 'RPM_MIN', 'RPM_MAX', 'PLANT_A', 'BAD_YIELD', 'label', 'features', 'rawPrediction', 'probability', 'prediction', 'predictedLabel'], 'values': [[87.113257, 87.290196, 87.017004, 65.000117, 64.846, 65.1839, 92.93055, 92.934958, 92.91834, 86.453545, 86.63842, 86.425383, 5010.837639, 5010.67, 5011.03, 0, 'GOOD', 0.0, [87.113257, 87.290196, 87.017004, 65.000117, 64.846, 65.1839, 92.93055, 92.934958, 92.91834, 86.453545, 86.63842, 86.425383, 5010.837639, 5010.67, 5011.03, 0.0], [0.0, 20.0], [0.0, 1.0], 1.0, 'BAD'], [91.628031, 91.591823, 91.57625, 29.999903, 29.9447, 30.055, 88.597187, 88.598325, 88.595852, 91.467674, 91.52794, 91.439629, 5008.691111, 5007.63, 5009.67, 0, 'GOOD', 0.0, [91.628031, 91.591823, 91.57625, 29.999903, 29.9447, 30.055, 88.597187, 88

In [113]:
subscription.get_details()

{'metadata': {'guid': '3437f090-aff0-49d9-b062-08b0550882fa',
  'url': '/v1/data_marts/8cb0facb-0ece-42f4-a78f-1ff08406b717/service_bindings/5a8c208d-b89f-4fcd-af26-c3cd4a48ac01/subscriptions/3437f090-aff0-49d9-b062-08b0550882fa',
  'created_at': '2019-06-06T14:18:04.150Z',
  'modified_at': '2019-06-06T14:18:09.676Z'},
 'entity': {'service_binding_id': '5a8c208d-b89f-4fcd-af26-c3cd4a48ac01',
  'asset_properties': {'runtime_environment': 'spark-2.3',
   'predicted_target_field': 'predictedLabel',
   'training_data_schema': {'type': 'struct',
    'fields': [{'name': 'WS_001_FLOW_MEAN',
      'type': 'double',
      'nullable': True,
      'metadata': {'modeling_role': 'feature'}},
     {'name': 'WS_001_FLOW_MIN',
      'type': 'double',
      'nullable': True,
      'metadata': {'modeling_role': 'feature'}},
     {'name': 'WS_001_FLOW_MAX',
      'type': 'double',
      'nullable': True,
      'metadata': {'modeling_role': 'feature'}},
     {'name': 'WS_001_CONC_MEAN',
      'type': 'dou

# Insert historical payloads

In [114]:
!rm payload_history*.json
!wget https://raw.githubusercontent.com/shadgriffin/propensitytobuylab/master/payload_history_1.json
!wget https://raw.githubusercontent.com/shadgriffin/propensitytobuylab/master/payload_history_2.json
!wget https://raw.githubusercontent.com/shadgriffin/propensitytobuylab/master/payload_history_3.json
!wget https://raw.githubusercontent.com/shadgriffin/propensitytobuylab/master/payload_history_4.json
!wget https://raw.githubusercontent.com/shadgriffin/propensitytobuylab/master/payload_history_5.json
!wget https://raw.githubusercontent.com/shadgriffin/propensitytobuylab/master/payload_history_6.json
!wget https://raw.githubusercontent.com/shadgriffin/propensitytobuylab/master/payload_history_7.json

--2019-06-06 14:18:56--  https://raw.githubusercontent.com/shadgriffin/propensitytobuylab/master/payload_history_1.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 199.232.8.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|199.232.8.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2588835 (2.5M) [text/plain]
Saving to: 'payload_history_1.json'

payload_history_1.j 100%[===================>]   2.47M  --.-KB/s    in 0.09s   

2019-06-06 14:18:57 (28.0 MB/s) - 'payload_history_1.json' saved [2588835/2588835]

--2019-06-06 14:18:57--  https://raw.githubusercontent.com/shadgriffin/propensitytobuylab/master/payload_history_2.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.48.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2588676 (2.5M) [text/plain]
Saving to: 'payload_histor

In [115]:
historyDays = 7
from ibm_ai_openscale.supporting_classes import PayloadRecord, Feature
import datetime
import time

In [116]:
data_mart_id = subscription.get_details()['metadata']['url'].split('/service_bindings')[0].split('marts/')[1]
print(data_mart_id)

8cb0facb-0ece-42f4-a78f-1ff08406b717


In [117]:
performance_metrics_url = 'https://api.aiopenscale.cloud.ibm.com' + subscription.get_details()['metadata']['url'].split('/service_bindings')[0] + '/metrics'
print(performance_metrics_url)

https://api.aiopenscale.cloud.ibm.com/v1/data_marts/8cb0facb-0ece-42f4-a78f-1ff08406b717/metrics


## Insert historical fairness metrics

In [118]:
!rm fairness_records.json
!wget https://raw.githubusercontent.com/shadgriffin/oglabworking/master/fairness_records.json
import random

--2019-06-06 14:19:03--  https://raw.githubusercontent.com/shadgriffin/oglabworking/master/fairness_records.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.48.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6706 (6.5K) [text/plain]
Saving to: 'fairness_records.json'

fairness_records.js 100%[===================>]   6.55K  --.-KB/s    in 0s      

2019-06-06 14:19:03 (97.6 MB/s) - 'fairness_records.json' saved [6706/6706]



In [119]:

token_data = {
    'grant_type': 'urn:ibm:params:oauth:grant-type:apikey',
    'response_type': 'cloud_iam',
    'apikey': AIOS_CREDENTIALS['apikey']
}

response = requests.post('https://iam.bluemix.net/identity/token', data=token_data)
iam_token = response.json()['access_token']
iam_headers = {
    'Content-Type': 'application/json',
    'Authorization': 'Bearer %s' % iam_token
}

with open('fairness_records.json', 'r') as history_file:
    payloads = json.load(history_file)

for day in range(historyDays):
    print('Day', day + 1)
    for hour in range(24):
        score_time = (datetime.datetime.utcnow() + datetime.timedelta(hours=(-(24*day + hour + 1)))).strftime('%Y-%m-%dT%H:%M:%SZ')
        
        qualityMetric = {
            'metric_type': 'fairness',
            'binding_id': binding_uid,
            'timestamp': score_time,
            'subscription_id': model_uid,
            'asset_revision': model_uid,
            'deployment_id': deployment_uid,
            'value': random.choice(payloads)
        }

        response = requests.post(performance_metrics_url, json=[qualityMetric], headers=iam_headers)
print('Finished')

Day 1
Day 2
Day 3
Day 4
Day 5
Day 6
Day 7
Finished


## Insert historical quality metrics

In [120]:
token_data = {
    'grant_type': 'urn:ibm:params:oauth:grant-type:apikey',
    'response_type': 'cloud_iam',
    'apikey': AIOS_CREDENTIALS['apikey']
}

response = requests.post('https://iam.bluemix.net/identity/token', data=token_data)
iam_token = response.json()['access_token']
iam_headers = {
    'Content-Type': 'application/json',
    'Authorization': 'Bearer %s' % iam_token
}

measurements = [0.94, 0.91, 0.78, 0.82, 0.90, 0.94, 0.93]
for day in range(historyDays):
    print('Day', day + 1)
    for hour in range(24):
        score_time = (datetime.datetime.utcnow() + datetime.timedelta(hours=(-(24*day + hour + 1)))).strftime('%Y-%m-%dT%H:%M:%SZ')
        
        qualityMetric = {
            'metric_type': 'quality',
            'binding_id': binding_uid,
            'timestamp': score_time,
            'subscription_id': model_uid,
            'asset_revision': model_uid,
            'deployment_id': deployment_uid,
            'value': {
                'quality': measurements[day],
                'threshold': 0.85,
                'metrics': [
                    {
                        'name': 'auroc',
                        'value': measurements[day],
                        'threshold': 0.75
                    }
                ]
            }
        }

        response = requests.post(performance_metrics_url, json=[qualityMetric], headers=iam_headers)
print('Finished')

Day 1
Day 2
Day 3
Day 4
Day 5
Day 6
Day 7
Finished


## Insert historical performance metrics

In [121]:
token_data = {
    'grant_type': 'urn:ibm:params:oauth:grant-type:apikey',
    'response_type': 'cloud_iam',
    'apikey': AIOS_CREDENTIALS['apikey']
}

response = requests.post('https://iam.bluemix.net/identity/token', data=token_data)
iam_token = response.json()['access_token']
iam_headers = {
    'Content-Type': 'application/json',
    'Authorization': 'Bearer %s' % iam_token
}

for day in range(historyDays):
    print('Day', day + 1)
    for hour in range(24):
        score_time = (datetime.datetime.utcnow() + datetime.timedelta(hours=(-(24*day + hour + 1)))).strftime('%Y-%m-%dT%H:%M:%SZ')
        score_count = random.randint(600, 6000)
        score_resp = random.uniform(600, 3000)

        performanceMetric = {
            'metric_type': 'performance',
            'binding_id': binding_uid,
            'timestamp': score_time,
            'subscription_id': model_uid,
            'asset_revision': model_uid,
            'deployment_id': deployment_uid,
            'value': {
                'response_time': score_resp,
                'records': score_count
            }
        }

        response = requests.post(performance_metrics_url, json=[performanceMetric], headers=iam_headers)
print('Finished')

Day 1
Day 2
Day 3
Day 4
Day 5
Day 6
Day 7
Finished


## Configure Explainability

In [122]:
pd_data.head()

,WS_001_FLOW_MEAN,WS_001_FLOW_MIN,WS_001_FLOW_MAX,WS_001_CONC_MEAN,WS_001_CONC_MIN,WS_001_CONC_MAX,DMW_FLOW_MEAN,DMW_FLOW_MIN,DMW_FLOW_MAX,ALK_FLOW_MEAN,ALK_FLOW_MIN,ALK_FLOW_MAX,RPM_MEAN,RPM_MIN,RPM_MAX,PLANT_A,BAD_YIELD
0,87.226264,87.462261,87.192360,62.998835,62.8255,63.1551,92.682929,92.687150,92.67134,87.957784,88.105276,87.929657,5010.885417,5010.67,5011.11,0,BAD
1,87.061707,86.429871,87.104682,64.899764,60.0440,65.0691,92.930550,92.934958,92.91834,87.847303,87.127372,87.929657,4997.844915,4319.54,5009.69,0,BAD
13,87.230540,87.462261,87.192360,64.999974,64.8091,65.1523,92.930550,92.934958,92.91834,87.957784,88.105276,87.929657,5009.466250,5008.93,5009.94,0,BAD
16,87.085769,87.204164,87.017004,65.001700,64.7714,65.2815,92.930550,92.934958,92.91834,86.453545,86.638420,86.425383,5008.098264,5002.25,5010.52,0,BAD
30,87.213436,87.376229,87.104682,64.998859,64.8578,65.1560,92.930550,92.934958,92.91834,86.453545,86.638420,86.425383,5010.489097,5010.30,5010.75,0,BAD


In [123]:
from ibm_ai_openscale.supporting_classes import *
subscription.explainability.enable(training_data=df_training)

In [124]:
subscription.explainability.get_details()

{'enabled': True,
 'parameters': {'training_statistics': {'mins': {'12': 4546.895,
    '8': 88.59585200000002,
    '4': 7.0,
    '11': 86.425383,
    '9': 86.45354499999998,
    '13': 3553.7,
    '5': 26.0,
    '10': 86.63842,
    '6': 88.59549100000002,
    '1': 86.085741,
    '14': 4652.0,
    '0': 86.611007,
    '2': 86.578615,
    '7': 88.47442099999998,
    '3': 26.0},
   'categorical_columns': ['PLANT_A'],
   'feature_values': {'12': [3, 0, 1, 2],
    '8': [1, 0, 2],
    '4': [2, 1, 3, 0],
    '15': [0, 1],
    '11': [0, 1, 2],
    '9': [0, 1],
    '13': [3, 0, 1, 2],
    '5': [2, 1, 3, 0],
    '10': [0, 1],
    '6': [1, 0, 2],
    '1': [1, 0, 3, 2],
    '14': [3, 1, 2, 0],
    '0': [1, 0, 3, 2],
    '2': [0, 3, 2, 1],
    '7': [1, 0],
    '3': [2, 0, 3, 1]},
   'd_maxs': {'12': [5007.904236, 5009.439306, 5010.242153, 5011.464514],
    '8': [88.59585200000002, 92.91834, 95.388333],
    '4': [29.934175, 29.9542, 64.77405, 64.9677],
    '11': [87.92965699999998, 91.439629, 92.44247

## Run fairness monitor

Kick off a fairness monitor run on current data. Depending on how fast the monitor runs, the table may not contain the most recent results.

In [125]:
run_details = subscription.fairness_monitoring.run()

In [126]:
subscription.fairness_monitoring.show_table()

2019-06-06 13:19:04+00:00,PLANT_A,"[0, 0]",False,0.98,86.5,5a8c208d-b89f-4fcd-af26-c3cd4a48ac01,3437f090-aff0-49d9-b062-08b0550882fa,3437f090-aff0-49d9-b062-08b0550882fa,2b37af4c-7b67-41ff-9403-1b6d33a5c755,


## Additional data to help debugging

In [127]:
#print('Datamart:', data_mart_id)
print('Model:', model_uid)
print('Deployment:', deployment_uid)
print('Binding:', binding_uid)
print('Scoring URL:', propensity_to_buy_scoring_endpoint)

Model: 3437f090-aff0-49d9-b062-08b0550882fa
Deployment: 2b37af4c-7b67-41ff-9403-1b6d33a5c755
Binding: 5a8c208d-b89f-4fcd-af26-c3cd4a48ac01
Scoring URL: https://us-south.ml.cloud.ibm.com/v3/wml_instances/5a8c208d-b89f-4fcd-af26-c3cd4a48ac01/deployments/2b37af4c-7b67-41ff-9403-1b6d33a5c755/online


## Identify transactions for Explainability

Transaction IDs identified by the cells below can be copied and pasted into the Explainability tab of the OpenScale dashboard.

In [128]:
import json, random

DEPLOYMENT_NAME = "Yield Model"
MIN_RECORDS = 1000
MAX_RECORDS = 1000

In [129]:
!rm df_payload_biased.json
!wget https://raw.githubusercontent.com/shadgriffin/oglabworking/master/df_payload_biased.json

--2019-06-06 14:19:46--  https://raw.githubusercontent.com/shadgriffin/oglabworking/master/df_payload_biased.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.48.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 290338 (284K) [text/plain]
Saving to: 'df_payload_biased.json'

df_payload_biased.j 100%[===================>] 283.53K  --.-KB/s    in 0.01s   

2019-06-06 14:19:47 (21.3 MB/s) - 'df_payload_biased.json' saved [290338/290338]



In [130]:
wml_deployments = wml_client.deployments.get_details()
scoring_url = None
for deployment in wml_deployments['resources']:
    if DEPLOYMENT_NAME == deployment['entity']['name']:
        scoring_url = deployment['entity']['scoring_url']
        break
    
print("Scoring URL: {}".format(scoring_url))

Scoring URL: https://us-south.ml.cloud.ibm.com/v3/wml_instances/5a8c208d-b89f-4fcd-af26-c3cd4a48ac01/deployments/2b37af4c-7b67-41ff-9403-1b6d33a5c755/online


In [131]:
try:
    with open('df_payload_biased.json', 'r') as scoring_file:
        scoring_data = json.load(scoring_file)
    print('file found')
    
except:
    !wget https://raw.githubusercontent.com/shadgriffin/oglabworking/master/df_payload_biased.json
    with open('df_payload_biased.json', 'r') as scoring_file:
        scoring_data = json.load(scoring_file)
    print('file downloaded')


file found


In [132]:
fields = scoring_data['fields']
values = []
for _ in range(0, random.randint(MIN_RECORDS, MAX_RECORDS)):
    values.append(random.choice(scoring_data['values']))
payload_scoring = {"fields": fields, "values": values}

scoring_response = wml_client.deployments.score(scoring_url, payload_scoring)
print(scoring_response)

{'fields': ['WS_001_FLOW_MEAN', 'WS_001_FLOW_MIN', 'WS_001_FLOW_MAX', 'WS_001_CONC_MEAN', 'WS_001_CONC_MIN', 'WS_001_CONC_MAX', 'DMW_FLOW_MEAN', 'DMW_FLOW_MIN', 'DMW_FLOW_MAX', 'ALK_FLOW_MEAN', 'ALK_FLOW_MIN', 'ALK_FLOW_MAX', 'RPM_MEAN', 'RPM_MIN', 'RPM_MAX', 'PLANT_A', 'BAD_YIELD', 'label', 'features', 'rawPrediction', 'probability', 'prediction', 'predictedLabel'], 'values': [[90.504072, 90.645465, 90.524116, 29.472222, 29.0, 30.0, 88.597187, 88.598325, 88.595852, 90.464848, 90.550036, 90.43678, 5009.537986, 5006.72, 5010.5, 1, 'GOOD', 0.0, [90.504072, 90.645465, 90.524116, 29.472222, 29.0, 30.0, 88.597187, 88.598325, 88.595852, 90.464848, 90.550036, 90.43678, 5009.537986, 5006.72, 5010.5, 1.0], [19.88170837616458, 0.11829162383542224], [0.9940854188082289, 0.005914581191771112], 0.0, 'GOOD'], [91.431339, 91.50579, 91.400894, 29.999394, 29.8884, 30.133, 88.597187, 88.598325, 88.595852, 91.467674, 91.52794, 91.439629, 4996.165556, 4992.8, 5002.19, 1, 'GOOD', 0.0, [91.431339, 91.50579,

In [133]:
time.sleep(10)
payload_data = subscription.payload_logging.get_table_content(limit=200)
payload_data.filter(items=['scoring_id', 'predictedLabel', 'probability','PLANT_A'])

,scoring_id,predictedLabel,probability,PLANT_A
0,3a627b113afd90c69879a6cca8c6e8aa-10,GOOD,"[0.9977547034125293, 0.0022452965874707907]",0
1,3a627b113afd90c69879a6cca8c6e8aa-100,GOOD,"[0.9932860405691477, 0.0067139594308523965]",1
2,3a627b113afd90c69879a6cca8c6e8aa-1000,GOOD,"[0.9949915655213477, 0.005008434478652235]",1
3,3a627b113afd90c69879a6cca8c6e8aa-101,GOOD,"[0.9747305545685082, 0.025269445431491795]",0
4,3a627b113afd90c69879a6cca8c6e8aa-102,GOOD,"[0.9886767466288677, 0.01132325337113225]",1
5,3a627b113afd90c69879a6cca8c6e8aa-103,GOOD,"[0.9933811082859318, 0.006618891714068147]",1
6,3a627b113afd90c69879a6cca8c6e8aa-104,BAD,"[0.0, 1.0]",0
7,3a627b113afd90c69879a6cca8c6e8aa-105,BAD,"[0.0, 1.0]",0
8,3a627b113afd90c69879a6cca8c6e8aa-106,BAD,"[0.3217695536445536, 0.6782304463554463]",0
9,3a627b113afd90c69879a6cca8c6e8aa-107,GOOD,"[0.9940906646204469, 0.005909335379553137]",0


## Congratulations!

You have finished the hands-on lab for IBM Watson OpenScale. You can now view the [OpenScale Dashboard](https://aiopenscale.cloud.ibm.com/). Click on the tile for the Propensity to Buy model to see fairness, accuracy, and performance monitors. Click on the timeseries graph to get detailed information on transactions during a specific time window.

